In [ ]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import glob
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_df = pd.DataFrame(columns=["image", "damage"])
train_df.image = [file for file in glob.glob('./Processed_Images/train/damage/*.jpeg')] + [file for file in glob.glob('./Processed_Images/train/no_damage/*.jpeg')]
train_df.damage = ["1"]*5000 + ["0"]*5000
train_df.reset_index(inplace=True, drop=True)

test_df = pd.DataFrame(columns=["image", "damage"])
test_df.image = [file for file in glob.glob('./Processed_Images/test/damage/*.jpeg')] + [file for file in glob.glob('./Processed_Images/test/no_damage/*.jpeg')]
test_df.damage = ["1"]*1000 + ["0"]*1000
test_df.reset_index(inplace=True, drop=True)

val_df = pd.DataFrame(columns=["image", "damage"])
val_df.image = [file for file in glob.glob('./Processed_Images/val/damage/*.jpeg')] + [file for file in glob.glob('./Processed_Images/val/no_damage/*.jpeg')]
val_df.damage = ["1"]*1000 + ["0"]*1000
val_df.reset_index(inplace=True, drop=True)

In [ ]:
train_data_gen = ImageDataGenerator(horizontal_flip=True)
train_data = train_data_gen.flow_from_dataframe(
    dataframe = train_df,
    x_col = "image",
    y_col = "damage",
    target_size = (128, 128),
    color_mode = "grayscale",
    class_mode = "binary",
    shuffle = True,
    subset = "training",
    batch_size = 100
)

test_data_gen = ImageDataGenerator(horizontal_flip=True)
test_data = test_data_gen.flow_from_dataframe(
    dataframe = test_df,
    x_col = "image",
    y_col = "damage",
    target_size = (128, 128),
    color_mode = "grayscale",
    class_mode = "binary",
    shuffle = True,
    batch_size = 100
)

val_data_gen = ImageDataGenerator(horizontal_flip=True)
val_data = val_data_gen.flow_from_dataframe(
    dataframe = val_df,
    x_col = "image",
    y_col = "damage",
    target_size = (128, 128),
    color_mode = "grayscale",
    class_mode = "binary",
    shuffle = True,
    batch_size = 100
)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=128, kernel_size=(5, 5), activation='relu', input_shape=(128, 128, 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(5, 5), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer = tf.optimizers.SGD(learning_rate=0.0005, momentum=0.8),
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(train_data, epochs=30, validation_data=val_data)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right', bbox_to_anchor=(1.3, 1))
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right', bbox_to_anchor=(1.3, 1))
plt.show()

In [ ]:
model.evaluate(test_data)